In [ ]:
# Made following John Mannelli's tutorial
# link here : https://jman4190.medium.com/build-your-own-spotify-wrapped-with-python-spotify-and-glide-apps-493dc7da20b

In [1]:
!pip install requests

In [2]:
!pip install spotipy

In [3]:
import requests
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import time
import datetime
from urllib.parse import urlencode

In [4]:
import pandas as pd
import getpass

In [5]:
SPOTIPY_REDIRECT_URI = 'http://127.0.0.1:9090'
SCOPE = 'user-top-read'  # asked in the documentation for authentification     

In [6]:
SPOTIPY_CLIENT_ID = getpass.getpass()

········


In [7]:
CLIENT_SECRET= getpass.getpass()

········


In [8]:
#initializing sp to access
sp=spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID,client_secret=CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI,scope=SCOPE))

In [11]:
#FUNCTION - get track ids to access the track features

def get_track_ids(time_frame):
    track_ids =[]   # empty list
    for song in time_frame['items']: #for each song we get in the dictionnary its item and within we get its id
        track_ids.append(song['id'])
    return track_ids

In [15]:
#FUNCTION - that retrieves the tracks' features (metadata)

def get_track_features(id):
    meta= sp.track(id)
    name= meta['name']
    album= meta['album']['name']
    artist= meta['album']['artists'][0]['name']
    spotify_url=meta['external_urls']['spotify']
    album_cover=meta['album']['images'][0]['url']
    track_info=[name, album, artist,spotify_url,album_cover] # let's put them into a new list
    return track_info
    

In [20]:
#CONNECTING WITH THE GOOGLESHEET 

In [21]:
!pip install gspread

In [22]:
import gspread

In [27]:
#FUNCTION - that inserts the tracks into the Googlesheet
def insert_to_gsheet(track_ids):
    tracks = [] #1
    for i in range(len(track_ids)):  #2 
        time.sleep(.5) #3
        track = get_track_features(track_ids[i]) #4
        tracks.append(track) #5

    df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'spotify_url', 'album_cover']) #6
    gc = gspread.service_account(filename=r'C:\Users\lucie\Desktop\IRONHACK\99_OTHER\01_spotify_wrapped\allwrappedup-spotify-project-885278e87a5a.json') #7
    sh = gc.open('wrapped_spotify_GS') #8
    worksheet = sh.worksheet(f'{time_period}') #9 
    worksheet.update([df.columns.values.tolist()] + df.values.tolist()) #10
    print('Done')

In [30]:
#1# in a new list
#2# loop over track ids
#3# allow a little time for the API
#4# get the info
#5# add them to the new list
#6# create a dataframe from the list
#7# authentication for the gsheet -the json file is created when you do your credential when you initialize the google API
#8# open the Googlesheet *** the spreadsheet is shared with the credentials you get from the json file --> share -> mail
#9# go over the 3 time period of spotify short medium long term 
#10# for the update

In [29]:
#CALLING THE FUNCTIONS 
time_ranges=['short_term','medium_term','long_term'] #spreadsheet named like the spotify's time period #to make things easier
for time_period in time_ranges:
    top_tracks=sp.current_user_top_tracks(limit=10,offset=0, time_range=time_period)
    track_ids=get_track_ids(top_tracks) #calling the function get_track_ids for the 10 songs
    insert_to_gsheet(track_ids)

Done
Done
Done
